In [1]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

import matplotlib.pyplot as plt

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix

import prepare_readme as p
import explore as e

In [2]:
df = p.prep_readme()

In [3]:
df.head()

,language,readme_contents,repo,readme_contents_stemmed,readme_contents_lemmatized
0,CSS,# Welcome\n\n> **Warning**: this book is **not...,/nakov/Practical-Cryptography-for-Developers-Book,welcom warn book finish still work chapter com...,welcome warning book finished still working ch...
1,Java,# Monorepo of Deeplearning4j\n\nWelcome to the...,/eclipse/deeplearning4j,monorepo deeplearn j welcom new monorepo deepl...,monorepo deeplearning j welcome new monorepo d...
2,Java,# H2O\n\n[![Join the chat at https://gitter.im...,/h2oai/h2o-3,h join chat http gitter im h oai h http badg g...,h join chat http gitter im h oai h http badge ...
3,C++,"<div align=""center"">\n <img src=""https://www....",/tensorflow/tensorflow,div align center img src http www tensorflow o...,div align center img src http www tensorflow o...
4,C++,gRPC - An RPC library and framework\n=========...,/grpc/grpc,grpc rpc librari framework grpc modern open so...,grpc rpc library framework grpc modern open so...


In [4]:
df_v = df.language.value_counts(dropna=False)

In [5]:
labels = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels

,n,percent
HTML,20,0.190476
C++,20,0.190476
Python,15,0.142857
JavaScript,10,0.095238
Shell,10,0.095238
C,10,0.095238
CSS,10,0.095238
Java,10,0.095238


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [17]:
# words that are programming languages only
#verterize ngrams

In [49]:
# restrict df to target and predicted veriables
df = df[['readme_contents_lemmatized','language']]

In [59]:
# vectorize stemmed readme contents and assign X any y veriables
tfidf = TfidfVectorizer(ngram_range=(1, 1),use_idf=False,sublinear_tf=True)
X = tfidf.fit_transform(df.readme_contents_lemmatized)
y = df.language

In [65]:
# # split data into test and train
# X_train, X_test, = train_test_split(X, y, stratify=y,test_size=.2)

In [72]:
def train_test_split(df):
    from sklearn.model_selection import train_test_split
    X = df.drop(columns=["language"])
    y = df.language
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8, random_state = 123, stratify=y)
    train = X_train.merge(y_train, left_index = True, right_index=True)
    test = X_test.merge(y_test, left_index=True, right_index=True)
    return train, test

In [73]:
train, test = train_test_split(df)

In [74]:
# create classifier object and fit it to the data
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)
clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [75]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.58


In [76]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 0.29


In [77]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
numeric_features = train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train.select_dtypes(include=['object']).drop(['language'], axis=1).columns
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),('cat', categorical_transformer, categorical_features)])
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifiers = [
    KNeighborsClassifier(12),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True, random_state = 123),
    DecisionTreeClassifier(min_samples_split=5, random_state=123),
    RandomForestClassifier(random_state=123),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_train, y_train))
    print("model score: %.3f" % pipe.score(X_test, y_test))

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [ ]:
def get_words(text):
    
    words = re.sub(r'[^\w\s]', '', text).split()
    return words

In [ ]:
def get_ngrams(n=2):

    ngram_master_list = []

    for row in df.readme_contents_stemmed.apply(get_words):

        bigrams = nltk.ngrams(row, n)

        ngram_master_list.extend(bigrams)
    
    return ngram_master_list

In [ ]:
def get_ngram_strings(ngrams):
    
    string_list = []

    for pair in ngrams:
    
        first = pair[0]

        second = pair[1]
    
        string_list.append(f"{first} {second}")
        
    return string_list

In [10]:
TfidfVectorizer()

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(2, 2))
X = tfidf.fit_transform()
y = df.language

In [ ]:
df['ngrams'] = e.get_ngrams(df,n=2)

In [ ]:
df.head(10)

In [ ]:
HTML_list =[]

for lst in df.readme_contents_stemmed.apply(p.tokenize)[df.language=='HTML']:
    
    HTML_list.extend(lst)
    
print(HTML_list)

In [ ]:
def tokenize(article):
    '''
    tokenizes words in a string
    '''

    # create token object
    tokenizer = nltk.tokenize.ToktokTokenizer()

    # use object to tokenize string
    article = tokenizer.tokenize(article, return_str=True)
    
    return article

#new svc or decition tree

In [ ]:
# create classifier object and fit it to the data
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)
clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')